In [ ]:
!rm -rf ./14mult_limit

In [ ]:
%config IPCompleter.greedy=True
import numpy as np
import tensorflow as tf
import time

In [ ]:
%run ./datagen.py
#datagen, (x_train, y_train), (x_test, y_test) = data_preparation()

datagen, (x_train, y_train), (x_test, y_test) = data_preparation()

In [ ]:
path_logdir = './binary_resnet/'
path_model = './binary_resnet/'
num_gpu = 5
batch_size = 100
iterations = x_train.shape[0] // (batch_size * num_gpu)
epochs = 800
old_acc = 0
start_lr = 1e-2
end_lr = 1e-3
decay_rate = (end_lr / start_lr) ** (1 / epochs)

In [ ]:
# %run ./binary_layer.py 
# # binary_dense: dense()
# # binary_conv2d: conv2d()

# def inference(inputs, is_train):

#     # L1: 2*128conv + pooling + bn
#     with tf.variable_scope("ConvBlock1"):
#         x = conv2d(inputs=inputs, 
#                    filters=128, 
#                    kernel_size=(3, 3),
#                    strides=(1, 1), 
#                    padding='same')
#         x = tf.layers.batch_normalization(x, training=is_train)
#         x = tf.nn.sigmoid(x)

#         x = conv2d(inputs=x,
#                    filters=128, 
#                    kernel_size=(3, 3), 
#                    strides=(1, 1), 
#                    padding='same')
#         x = 4 * tf.layers.average_pooling2d(x, pool_size=(2, 2), strides=(2, 2), padding='SAME')
#         x = tf.layers.batch_normalization(x, training=is_train)        
#         x = tf.nn.sigmoid(x)
# #         x = tf.layers.dropout(x, 0.1)

#     # L2: 2*256conv + pooling + bn + dropout
#     with tf.variable_scope("ConvBlock2"):
#         x = conv2d(inputs=x,
#                    filters=256, 
#                    kernel_size=(3, 3), 
#                    strides=(1, 1), 
#                    padding='same')
#         x = tf.layers.batch_normalization(x, training=is_train)        
#         x = tf.nn.sigmoid(x)

#         x = conv2d(inputs=x,
#                    filters=256, 
#                    kernel_size=(3, 3), 
#                    strides=(1, 1), 
#                    padding='same')
#         x = 4*tf.layers.average_pooling2d(x, pool_size=(2, 2), strides=(2, 2), padding='SAME')
#         x = tf.layers.batch_normalization(x, training=is_train)
#         x = tf.nn.sigmoid(x)
# #         x = tf.layers.dropout(x, 0.1)

#     # L3: 2*512conv + pooling + dropout
#     with tf.variable_scope("ConvBlock3"):
#         x = conv2d(inputs=x,
#                    filters=512, 
#                    kernel_size=(3, 3), 
#                    strides=(1, 1), 
#                    padding='same')
#         x = tf.layers.batch_normalization(x, training=is_train)
#         x = tf.nn.sigmoid(x)
        
#         x = conv2d(inputs=x,
#                    filters=512, 
#                    kernel_size=(3, 3), 
#                    strides=(1, 1), 
#                    padding='same')
#         x = 4*tf.layers.average_pooling2d(x, pool_size=(2, 2), strides=(2, 2), padding='SAME')
#         x = tf.layers.batch_normalization(x, training=is_train)
#         x = tf.nn.sigmoid(x)    
# #         x = tf.layers.dropout(x, 0.1)

#     with tf.variable_scope("Flatten"):
#         x = tf.transpose(x, perm=[0, 3, 1, 2])
#         x = tf.layers.flatten(x)

#     # L4: 2*FC1024 + bn + dropout
#     with tf.variable_scope("FCBlock1"):
#         x = dense(x, units=1024)
#         x = tf.layers.batch_normalization(x, training=is_train)       
#         x = tf.nn.sigmoid(x)

#         x = dense(x, units=1024)
#         x = tf.layers.batch_normalization(x, training=is_train)
#         x = tf.nn.sigmoid(x)   
# #         x = tf.layers.dropout(x, 0.1)

#     # predict layer
#     with tf.variable_scope("Prediction"):
#         pred = dense(x, units=10)
    
#     return pred

In [ ]:
%run ./binary_layer.py 

# resnet layer
def res_layer(inputs, filter_num, filter_size, stride, is_train, 
              conv_first=False, batch_norm=True, activation=True):
    
    x = inputs
    
    if conv_first:
        x = conv2d(inputs=x, filters=filter_num, 
                   kernel_size=filter_size, strides=stride, padding='same')
        if batch_norm:
            x = tf.layers.batch_normalization(x, training=is_train)
        if activation:
            x = tf.square(x)
    else:
        if batch_norm:
            x = tf.layers.batch_normalization(x, training=is_train)
        if activation:
            x = tf.square(x)
        x = conv2d(inputs=x, filters=filter_num, 
                   kernel_size=filter_size, strides=stride, padding='same')
    
    return x

In [ ]:
def resnetv2(inputs, is_train):
    
    with tf.variable_scope("Conv1"):
        x = tf.layers.conv2d(inputs=inputs, filters=16, 
                             kernel_size=3, strides=1, padding='same')
        x = tf.layers.batch_normalization(x, training=is_train)
        x = tf.nn.relu(x)
          
    # Res Blocks
    for stack in range(3):
        for block in range(6):
            with tf.variable_scope('ResBlock{}'.format(stack*6+block+1)):
                
                batch_norm = True
                activation = True
                stride = 1
                if stack == 0:
                    filter_num = 16
                    if block == 0:
                        batch_norm = False
                        activation = False
                else:
                    filter_num = 16*2*stack
                    if block == 0:
                        stride = 2
                
                residual_x = x
                with tf.variable_scope('conv1'):
                    x = res_layer(x, filter_num, 1, stride, is_train, 
                                  batch_norm=batch_norm, activation=activation)
                with tf.variable_scope('conv2'):
                    x = res_layer(x, filter_num, 3, 1, is_train, 
                                  batch_norm=batch_norm, activation=activation)
                with tf.variable_scope('conv3'):
                    x = res_layer(x, filter_num*4, 1, 1, is_train, 
                                  batch_norm=batch_norm, activation=activation)
                if block == 0:
                    with tf.variable_scope('residual'):
                        residual_x = res_layer(residual_x, filter_num*4, 1, stride, is_train, 
                                               batch_norm=False, activation=False)
                x = x + residual_x   
    
    with tf.variable_scope("AfterResBlock"):
        x = tf.layers.batch_normalization(x, training=is_train)                
        x = tf.nn.relu(x)
        x = tf.layers.average_pooling2d(x, pool_size=2, strides=2, padding='SAME', name='pool1')
    #x.shape = (?, 4, 4, 256)
    
    with tf.variable_scope("Flatten"):
        x = tf.transpose(x, perm=[0, 3, 1, 2])
        x = tf.layers.flatten(x)
    
    with tf.variable_scope("Prediction"):
        pred = tf.layers.dense(x, units=10)
        
    return pred

In [ ]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g, _ in grad_and_vars:
            expend_g = tf.expand_dims(g, 0)
            grads.append(expend_g)
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [ ]:
with tf.device("/cpu:0"):
    
    inputs = tf.placeholder(tf.float32, [None, 32, 32, 3], name='input')
    outputs = tf.placeholder(tf.float32, [None, 10], name='output')
    is_train = tf.placeholder(tf.bool, name='is_train')
    
    global_step = tf.Variable(0, trainable=False)
    l_r = tf.train.exponential_decay(start_lr, global_step, iterations, decay_rate, staircase=True)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        opt = tf.train.AdamOptimizer(learning_rate=l_r)

    tower_grads = []
    tower_acc = []
    tower_loss = []
    
    with tf.variable_scope(tf.get_variable_scope()) as variable_scope:
#         print('-'*20)
#         print(variable_scope.name)
#         print('-'*20)
        for i in range(num_gpu):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    x = inputs[i * batch_size:(i + 1) * batch_size]
                    y = outputs[i * batch_size:(i + 1) * batch_size]
                    pred = resnetv2(x, is_train)
                    
                    tf.get_variable_scope().reuse_variables()
                    
                    loss = tf.losses.softmax_cross_entropy(y, pred)
                    tower_loss.append(loss)
#                     loss = tf.reduce_mean(tf.nn.sigmoid(tf.losses.hinge_loss(y, pred)))
                    grads = opt.compute_gradients(loss)
                    tower_grads.append(grads)

                    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
                    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
                    tower_acc.append(accuracy)
    
    losses = tf.reduce_mean(tower_loss)
    grads = average_gradients(tower_grads)
    accs = tf.reduce_mean(tower_acc)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = opt.apply_gradients(grads, global_step=global_step)
    
    saver = tf.train.Saver(tf.global_variables())
    tf.summary.scalar('loss', losses)
    tf.summary.scalar('accuracy', accs)
    tf.summary.scalar('learning_rate', l_r)
    merged = tf.summary.merge_all()

In [ ]:
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:

    print('*****************Training Start!*****************')
    train_writer = tf.summary.FileWriter(path_logdir+'train', sess.graph)
    sess.run(tf.global_variables_initializer())

    for m in range(epochs):
        start = time.time()
        batch_gen = datagen.flow(x_train, y_train, batch_size=batch_size*num_gpu)

        for i in range(iterations):
            x_batch, y_batch = next(batch_gen)
            _, loss_train, summary = sess.run([train_op, losses, merged],
                                              {inputs: x_batch, outputs: y_batch, is_train: True})
            train_writer.add_summary(summary, m * iterations + i)
        
        val_accs = []
        for i in range(5000//(batch_size*num_gpu)):
            val_acc = sess.run(accs,{inputs: x_test[i*batch_size*num_gpu:(i+1)*num_gpu*batch_size], 
                                     outputs: y_test[i*batch_size*num_gpu:(i+1)*num_gpu*batch_size], 
                                     is_train: False})
            val_accs.append(val_acc)

        if np.mean(val_accs) > old_acc:
            old_acc = np.mean(val_accs)
            saver.save(sess, path_model+'cifar10.ckpt')

        end = time.time()
        print('Epoch: {}'.format(m + 1),
              'Train_loss: {:.3f}'.format(loss_train),
              'Val_acc: {:.3f}'.format(np.mean(val_accs)),
              'Time consumed: {:.4f} s'.format(end - start))

    print('*****************Training End!*****************')